# Person Detector #

This script is used to find the person in the image so that the person can be cropped out and be used in the project. This will create a more accurate result.

### Import dependencies ###

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from yolov3.utils import Load_Yolo_model, image_preprocess, postprocess_boxes, nms
from yolov3.configs import *
import os

### Import images ###

In [36]:
images = []
images_path = os.path.join("pose", "reference_front")

for image in os.listdir(images_path):
    f = os.path.join(images_path, image)

    if os.path.isfile(f):
        images.append(f)
        
print(images)

['pose\\reference_front\\a_hidari_gedan_barai.png', 'pose\\reference_front\\b_migi_chudan_oi_zuki.png', 'pose\\reference_front\\c_migi_gedan_barai.png', 'pose\\reference_front\\d_migi_tetsui_uchi.png', 'pose\\reference_front\\e_hidari_chudan_oi_zuki.png', 'pose\\reference_front\\f_hidari_gedan_barai.png', 'pose\\reference_front\\g_migi_jodan_age_uke.png', 'pose\\reference_front\\h_hidari_age_uke_jodan.png', 'pose\\reference_front\\i_migi_jodan_age_uke.png', 'pose\\reference_front\\j_hidari_gedan_barai.png', 'pose\\reference_front\\k_migi_chudan_oi_zuki.png', 'pose\\reference_front\\m_hidari_chudan_oi_zuki.png', 'pose\\reference_front\\n_hidari_gedan_barai.png', 'pose\\reference_front\\o_migi_chudan_oi_zuki.png', 'pose\\reference_front\\p_hidari_chudan_oi_zuki.png', 'pose\\reference_front\\q_migi_chudan_oi_zuki.png', 'pose\\reference_front\\r_hidari_chudan_shuto_uke.png', 'pose\\reference_front\\s_migi_chudan_shuto_uke.png', 'pose\\reference_front\\t_migi_chudan_shuto_uke.png', 'pose\\r

### Crop image function ###

In [9]:
def crop(image, bboxes):
    for bbox in bboxes:
        coor = np.array(bbox[:4], dtype=np.int32)
        class_ind = int(bbox[5])
        (x1, y1), (x2, y2) = (coor[0], coor[1]), (coor[2], coor[3])

        # crop image if person
        if class_ind == 0:
            cropped_image = image[y1-50:y2+50, x1-50:x2+50]
            return cropped_image
        
    return None

In [10]:
def detect_and_crop_image(Yolo, image_path, output_path, input_size=416, show=False, score_threshold=0.3, iou_threshold=0.45):
    original_image = cv2.imread(image_path)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    image_data = image_preprocess(np.copy(original_image), [
                                  input_size, input_size])
    image_data = image_data[np.newaxis, ...].astype(np.float32)

    if YOLO_FRAMEWORK == "tf":
        pred_bbox = Yolo.predict(image_data)
    elif YOLO_FRAMEWORK == "trt":
        batched_input = tf.constant(image_data)
        result = Yolo(batched_input)
        pred_bbox = []
        for key, value in result.items():
            value = value.numpy()
            pred_bbox.append(value)

    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)

    bboxes = postprocess_boxes(
        pred_bbox, original_image, input_size, score_threshold)
    bboxes = nms(bboxes, iou_threshold, method='nms')

    image = crop(original_image, bboxes)
    # CreateXMLfile("XML_Detections", str(int(time.time())), original_image, bboxes, read_class_names(CLASSES))

    if output_path != '':
        cv2.imwrite(output_path, image)
    if show:
        # Show the image
        cv2.imshow("Predicted image", image)
        # Load and hold the image
        cv2.waitKey(0)
        # To close the window after the required kill value was provided
        cv2.destroyAllWindows()

    return image

### Load yolo model ###

In [11]:
yolo = Load_Yolo_model()

### Detect and crop images ###

In [37]:
for image in images:
    path = image.split('\\')[2]
    detect_and_crop_image(yolo, image, f'./cropped_pose/reference_front/{path}', show=False)

1/1 [==============================] - 1s 577ms/step


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:783: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
